# A Script to Generate Resistor String Spice Files

**Yohanes Stefanus**

In [22]:
import numpy as np
import pandas as pd

dev_prefix = 'sky130_fd_pr__res_xhigh_po_'
dev_w = ['0p35', '0p69', '1p41', '2p85', '5p73']

df = pd.read_csv('Lcomp_data4.txt', sep=r'\s+')
#df = pd.read_csv('L_data6.txt', sep=r'\s+')

In [23]:
lines = []
node = 0

for seg_num in sorted(df['Segment'].unique()):
    segment_df = df[df['Segment'] == seg_num].reset_index(drop=True)
    num_res = len(segment_df)
    num_nodes = num_res + 1

    #subckt line
    port_list = [f"v{i}" for i in range(num_nodes)]
    subckt_line = f".subckt rseg_{int(seg_num)}_v3 " + " ".join(port_list) + " gnd"

    #device instance lines
    lines = [subckt_line]
    node = 0

    #randomness
    #np.random.seed(0)
    #variation = np.random.uniform(-0.005, 0.005, size=len(df))
    #df['L_varians'] = df['L'] * (1 + variation)

    for idx, row in segment_df.iterrows():
        l_val = f"{row['L']:.4f}"
        w_val = row['W']
        w_str = f"{float(w_val):.2f}".replace('.','p')

        if w_str not in dev_w:
            raise ValueError(f"Unsupported width '{w_str}' for row {idx}!")
        
        model_name  = dev_prefix + w_str

        line = f"XR{idx+1} v{node} v{node+1} gnd {model_name} L={l_val} mult=1 m=1"
        lines.append(line)
        node += 1
    
    lines.append(".ends")


    with open(f"../spice/rseg_{int(seg_num)}.spice", "w") as f:
        f.write("\n".join(lines))

In [15]:
print(np.linspace(1,-1,10))
np.random.seed(0)
variation = np.random.uniform(-0.005, 0.005, 10)
print(variation)


[ 1.          0.77777778  0.55555556  0.33333333  0.11111111 -0.11111111
 -0.33333333 -0.55555556 -0.77777778 -1.        ]
[ 0.00048814  0.00215189  0.00102763  0.00044883 -0.00076345  0.00145894
 -0.00062413  0.00391773  0.00463663 -0.00116558]
